# Test Attractor Dynamics with LIF Neurons

In [1]:
# Imports and funcs
%matplotlib widget
from brian2 import *
import numpy as np
import scipy.sparse as sp
import excitation_schedule as es
import brian_weight_submatrix as bws
import diagonal_sums as ds
import spikes_to_pcs as stp
import time
import pickle

In [32]:
start_scope()
# The following line suppresses a warning about order of executions in the abstract code: "v_post = clip(v_post + w, v_gaba, 0)"
# As far as I can tell, the warning is due to the inability of the OOE checker to deal with the "clip()" function, and is ok in this case
BrianLogger.suppress_hierarchy('brian2.codegen')

seed(seed=1)

################## INDEPENDENT PARAMETERS ################
# Node parameters
t_mem = 10*ms
t_adapt_e = 100*ms
t_adapt_i = 100*ms
t_refract_e = 6.3*ms       # 6.3 ± 1.7 Raastad 2003
t_refract_std_e = 1.7*ms
t_refract_i = 5*ms # jittered with std=1 below
v_rest = -70*mV
v_adapt_step_e = 6*mV # 4
v_adapt_step_i = 5*mV # 4
v_thresh = -50*mV
v_reset = -65*mV
v_gaba = -80*mV

# Network parameters
net_size = 15.0*cm # circumference of toroidal network - brain width: mouse=1cm, human=15cm
nxe = 100 #100
nye = 100 #100
nxi = 40 #40
nyi = 40 #40
p0ee = 0.8
lradee = 30 # in units of neuron spacing
p0ei = 0.4
lradei = 2 # in units of neuron spacing
p0ie = 0.3
lradie = 5 # [5] in units of neuron spacing
wbase_ee = 0.0
wrange_ee = 0.2 # 0.5
wbase_ei = 3.0
wrange_ei = 6.0
wbase_ie = -4.0
wrange_ie = -5.0
delay_ee = True
dendritic_delay_min = 2.0 * ms # 3ms from Jarsky NatNeuro 2005
dendritic_delay_range = 2.0 * ms
conduction_velocity = 5.0 * meter / second

# STDP parameters
t_stdppre = 20.0 * ms
t_stdppost = 20.0 * ms
dApre = 0.1245 * mV # 0.3 goes into short oscillation at d=.25
STDP_neg_pos_ratio = 1.05 # 1.05 Greater than 1 favors decay of weights
wmax = 5.0 * mV # Need about 20mV to trigger a spike
STDP_delayed = 0.0 # real time switch

# Stimulus parameters
attractor_size = 500
attractor_type = 'random' # 'random' or 'circular'
stim_freq = 8.0 # Hz
stim_duty = 0.5 # 0.25
stim_time = 1100 * ms # 1100
stim_dt = 1 * ms
stim_rate = 200 * Hz # 200
stim_ramp_on = 0.1
stim_ramp_off = 0.1
restim_delay = 0 * ms # 200
restim_time = 10 * ms # 10
restim_runtime = 1000 * ms # 200
restim_fraction = 0.25 # 0.2
restim_density = 1.0 # 1.0
restim_N = 2

# Timing parameters
defaultclock.dt = 0.01*ms

################## CALCULATED PARAMETERS ################
# Network geometry
Ne = nxe * nye
Ni = nxi * nyi
dxe = net_size / nxe
dye = net_size / nye
dxi = net_size / nxi
dyi = net_size / nyi

# STDP
dApost = -dApre * t_stdppre / t_stdppost * STDP_neg_pos_ratio

########################## ATTRACTOR ########################
# Set attractor nodes
attractor_nodes = np.random.choice(Ne,attractor_size,replace=False)
if attractor_type == 'circular':
    attractor_nodes = np.sort(attractor_nodes)
attractor_index = {e:i for i,e in enumerate(attractor_nodes)} # lookup dictionary for getting attractor index from node number

########################## STIMULUS ########################
# Set stimulus
ex = es.excitation_schedule(attractor_size,'circular',stim_dt/ms,stim_time/ms,stim_duty,stim_freq,ramp_on=stim_ramp_on,ramp_off=stim_ramp_off)
ex = np.vstack((ex,np.zeros((int(restim_delay/stim_dt),attractor_size))))
stimulus_teach = TimedArray(ex*stim_rate,dt=stim_dt)

restim = np.zeros((int(restim_runtime/stim_dt),attractor_size))
restim_nodes = np.random.choice(int(attractor_size * restim_fraction),int(attractor_size * restim_fraction * restim_density))
restim[:int(restim_time/stim_dt),restim_nodes] = 1.
stimulus_test = TimedArray(restim*stim_rate,dt=stim_dt)

########################## MODEL ########################
# E nodes decay towards v_rest-v_adapt, and v_adapt decays towards 0
eqs_e = '''
dv/dt = (v_rest-v-v_adapt)/t_mem : volt (unless refractory)
dv_adapt/dt = (-v_adapt)/t_adapt_e : volt
x : meter
y : meter
net_size : meter
refract : second
'''
reset_e = '''
v = v_reset
v_adapt += v_adapt_step_e
'''

# I nodes decay towards v_rest-v_adapt, and v_adapt decays towards 0
eqs_i = '''
dv/dt = (v_rest-v-v_adapt)/t_mem : volt (unless refractory)
dv_adapt/dt = (-v_adapt)/t_adapt_i : volt
x : meter
y : meter
net_size : meter
refract : second
'''
reset_i = '''
v = v_reset
v_adapt += v_adapt_step_i
'''

# STDP Synapse equations
# STDP_delayed is now a real time switch
STDP_eqs = '''
w : volt
dApre/dt = -Apre / t_stdppre : volt (event-driven)
dApost/dt = -Apost / t_stdppost : volt (event-driven)
dw : volt
'''
STDP_onpre = '''
v_post += w
Apre += dApre
dw += STDP_delayed * Apost
w = clip(w + (1.0-STDP_delayed) * Apost, 0, wmax)
'''
STDP_onpost = '''
Apost += dApost
dw += STDP_delayed * Apre
w = clip(w + (1.0-STDP_delayed) * Apre, 0, wmax)
'''

# Lorentz connection probability (mod (%) stuff makes it toroidal)
pLorentz = 'p0 / (1 + (((x_pre-x_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2 \
                     + ((y_pre-y_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2) / (lrad)**2)'

# Calculated conduction delay (mod (%) stuff makes it toroidal)
conduction_delay = 'dendritic_delay_min + rand() * dendritic_delay_range \
                        + sqrt( (((x_pre-x_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2 \
                        + ((y_pre-y_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2) ) / conduction_velocity'

###################### CONSTRUCT NETWORK ####################
# Generate nodes
PGteach = PoissonGroup(attractor_size, rates='stimulus_teach(t,i)',name='PGteach')
PGtest  = PoissonGroup(attractor_size, rates='stimulus_test(t,i)',name='PGtest')
E = NeuronGroup(nxe * nye, eqs_e, threshold='v>v_thresh', reset=reset_e, refractory='refract', method='euler', name='E')
I = NeuronGroup(nxi * nyi, eqs_i, threshold='v>v_thresh', reset=reset_i, refractory='refract', method='euler', name='I')

# set neuron locations (zero indexed, so net is in quadrant 1)
E.x = '(i % nxe) * dxe'
E.y = '(i // nxe) * dye'
I.x = '(i % nxi) * dxi'
I.y = '(i // nxi) * dyi'
E.net_size = net_size
I.net_size = net_size

# Set refractory periods with jitter
E.refract = t_refract_e + t_refract_std_e * randn(len(E))
I.refract = t_refract_i * (1.0 + 0.2 * randn(len(I)))

# Connect Poisson Groups to E
SPGteach = Synapses(PGteach, E, on_pre='v_post += 50*mV') # 50 guarantees firing of post
SPGteach.connect(i=np.arange(attractor_size),j=attractor_nodes)
SPGtest  = Synapses(PGtest, E, on_pre='v_post += 50*mV') # 50 guarantees firing of post
SPGtest.connect(i=np.arange(attractor_size),j=attractor_nodes)


# Connect EE, EI, IE
# No self connections in EE network
SEE = Synapses(E, E, STDP_eqs, on_pre=STDP_onpre, on_post=STDP_onpost, name='EE')
SEE.variables.add_constant('p0',p0ee)
SEE.variables.add_constant('lrad',lradee * net_size / nxe) # Convert from neuron spacing units to meters here
SEE.connect(condition = 'i != j', p=pLorentz)
SEE.w = '(wbase_ee + wrange_ee*rand())*mV'
if delay_ee == True:
    SEE.delay = conduction_delay

SEI = Synapses(E, I, 'w : volt', on_pre='v_post += w', name='EI')
SEI.variables.add_constant('p0',p0ei)
SEI.variables.add_constant('lrad',lradei * net_size / nxi) # Convert from neuron spacing units to meters here
SEI.connect(p=pLorentz)
SEI.w = '(wbase_ei + wrange_ei*rand())*mV'

SIE = Synapses(I, E, 'w : volt', on_pre='v_post = clip((v_post + w), v_gaba, 0)', name='IE')
SIE.variables.add_constant('p0',p0ie)
SIE.variables.add_constant('lrad',lradie * net_size / nxe) # Convert from neuron spacing units to meters here
SIE.connect(p=pLorentz)
SIE.w = '(wbase_ie + wrange_ie*rand())*mV'

########################## GO ########################
# Monitor stuff
MSE = SpikeMonitor(E,name='Espikemon')
# MSI = SpikeMonitor(I,name='Ispikemon')
# MVE = StateMonitor(E,'wsum',True,dt=max(defaultclock.dt,1.*ms),name='Estatemon')
# MVI = StateMonitor(I,'v',True,dt=max(defaultclock.dt,1.*ms),name='Istatemon')

# Initialize nodes at v_rest
E.v = v_rest
I.v = v_rest
store('initial_state')

wts_in_attractor = bws.brian_weights_in_subset(SEE,attractor_nodes,attractor_nodes)

# Break up run into little pieces to grab the sum of the weight matrix as we go
STDP_delayed = 1.0
PGtest.active = False # only teacher for now
wdt = 10*ms
wsumrec = []
wrec = []
for it in range(int(stim_time/wdt)):
    run(wdt)
    wsumrec.append(np.sum(SEE.w[wts_in_attractor]/mV))

dw0 = 1.0 * SEE.dw # Multiply by 1.0 to force copy
restore('initial_state')
SEE.w = clip(SEE.w + dw0,0,wmax)
SEE.dw = 0

wrec.append(bws.brian_weight_submatrix(SEE,attractor_nodes,attractor_nodes))
print(0, end = " ")

PGteach.active = False
PGtest.active = True
STDP_delayed = 0.0
for istep in range(restim_N):
    for it in range(int(restim_runtime/wdt)):
        run(wdt)
        wsumrec.append(np.sum(SEE.w[wts_in_attractor]/mV))
    wrec.append(bws.brian_weight_submatrix(SEE,attractor_nodes,attractor_nodes))
    if istep==0:
#        dApost = -dApre
        dApost *= 2
        dApre *= 2

    print(istep+1, end = " ")


0 1 2 

In [37]:
spikes_in_attractor = np.isin(MSE.i,attractor_nodes)
z = MSE.i[spikes_in_attractor]
spikes = np.ndarray(z.shape)
for node in attractor_index:
    spikes[z==node] = attractor_index[node]
tspike = MSE.t[spikes_in_attractor]/ms

figure(figsize=(8,8))
subplot(221)
plot(tspike, spikes, 'k.',markersize=1)

subplot(222)
#wts = SEE.w/mV
#hist(wts[wts>1], 20)
for it in range(restim_N+1):
    plot(np.arange(attractor_size)-attractor_size/2,ds.diagonal_sums(wrec[it]))

subplot(223)
imshow(wrec[0],vmax=3)
#plot(wsumrec)
 
subplot(224)
imshow(wrec[2],vmax=3)
tight_layout()

WARNING    <ipython-input-37-235ce8cfc826>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure(figsize=(8,8))
 [py.warnings]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
mylist = [tspike,spikes]
with open('chaos_spikes.pkl', 'wb') as f:
    pickle.dump(mylist, f)

In [2]:
with open('chaos_spikes.pkl', 'rb') as f:
    mylist = pickle.load(f)
tspike = mylist[0]
spikes = mylist[1]

In [41]:
figure(figsize=(10,5))

idx = (tspike>500) & (tspike<1000)
projections = stp.spikes_to_pcs(tspike[idx],spikes[idx],0.1,20,6)
subplot(121)
plot(projections[0][:,0],projections[0][:,2])
title('plasticity on')

ST=MSE.spike_trains()
ISI = []
allISI = np.array(0)
for i in range(attractor_size):
    st = ST[attractor_nodes[i]]/ms
    idx = (st>500) & (st<1500)
    train = st[idx]
    ISI.append(train[1:]-train[:-1])
    if i==0:
        allISI = np.asarray(train[1:]-train[:-1])
    else:
        allISI = np.concatenate((allISI,np.asarray(train[1:]-train[:-1])))

subplot(122)
allISI = allISI[allISI<42]
allISI = allISI[allISI>38]
hist(allISI,400)

WARNING    <ipython-input-41-bfb8ccc419c2>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure(figsize=(10,5))
 [py.warnings]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

PCA: explained variance ratio:  [0.36083653 0.35054917 0.12228152 0.11798827 0.0136867  0.01300519]


(array([  1.,   3.,   2.,   0.,   0.,   2.,   1.,   0.,   2.,   3.,   4.,
          1.,   1.,   0.,   3.,   2.,   3.,   4.,   5.,   1.,   4.,   1.,
          3.,   4.,   5.,   3.,   6.,   1.,   2.,   4.,   1.,   3.,   1.,
          2.,   5.,   0.,   5.,   6.,   4.,   6.,   3.,   9.,   1.,   4.,
          3.,   2.,   6.,   3.,   7.,   4.,   2.,   5.,   2.,   2.,   3.,
          2.,   7.,   0.,   4.,   7.,   5.,   3.,   7.,   2.,   7.,   7.,
          6.,  15.,  10.,   6.,   8.,  13.,   6.,   8.,   9.,   8.,  12.,
         13.,   9.,   9.,  10.,   8.,  17.,  10.,  11.,   8.,  12.,  12.,
         21.,   7.,  13.,  17.,  12.,  16.,  16.,  17.,  18.,  16.,  16.,
         15.,  22.,  19.,  22.,  11.,  24.,  18.,  20.,  33.,  22.,  25.,
         27.,  33.,  31.,  20.,   0.,  22.,  46.,  31.,  34.,  43.,  39.,
         44.,  32.,  37.,  22.,  44.,  45.,  46.,  56.,  34.,  50.,  51.,
         45.,  44.,  46.,  69.,  49.,  60.,  67.,  65.,  48.,  65.,  88.,
         80.,  66.,  71.,  70.,  64., 

In [17]:
figure(figsize=(10,5))

idx = (tspike>1500) & (tspike<2500)
projections = stp.spikes_to_pcs(tspike[idx],spikes[idx],0.1,20,5)
subplot(121)
plot(projections[0][:,0],projections[0][:,1])
title('plasticity on')

idx = (tspike>3500) & (tspike<4500)
projections = stp.spikes_to_pcs(tspike[idx],spikes[idx],0.1,20,5)
subplot(122)
plot(projections[0][:,0],projections[0][:,1])
title('plasticity off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

PCA: explained variance ratio:  [0.20989311 0.12692432 0.09856874 0.08559809 0.07766186]
PCA: explained variance ratio:  [0.0319646  0.02902332 0.02851587 0.0265619  0.02612815]


Text(0.5, 1.0, 'plasticity off')

In [3]:
attractor_indices_sorted = np.argsort(attractor_nodes)
attractor_sorted = attractor_nodes[attractor_indices_sorted]
spikes_in_attractor = np.isin(MSE.i,attractor_nodes)
idx_in_sort = np.searchsorted(attractor_sorted,MSE.i[spikes_in_attractor])
time = MSE.t[spikes_in_attractor]/ms
spikes = attractor_indices_sorted[idx_in_sort]

figure(figsize=(12,12))
subplot(331)
plot(time, spikes, 'k.',markersize=2)

subplot(332)
wts = SEE.w/mV
hist(wts[wts>1], 20)

subplot(333)
imshow(bws.brian_weight_submatrix(SEE,attractor_nodes,attractor_nodes))

nbin = 800
real,timeline = np.histogram(time,bins=nbin,weights=np.cos(2.0*np.pi*spikes/500))
imag,timeline = np.histogram(time,bins=nbin,weights=np.sin(2.0*np.pi*spikes/500))
cplx = real + 1.j * imag

freq = 1000. * (angle(cplx[1:] / cplx[:-1])) / (2. * np.pi * (timeline[1]-timeline[0]))
from scipy.signal import savgol_filter
fhat = savgol_filter(freq, 21, 2) # window size 51, polynomial order 3

subplot(336)
plot(timeline[1:-1],freq)
plot(timeline[1:-1],fhat,'r')

subplot(339)
plot(real,imag)
scatter(real,imag,c=range(nbin),cmap='jet')

subplot(334)
imshow(MVE.v / mV,aspect='auto')
subplot(335)
plot(MSE.t/ms, MSE.i, 'ko')
plot(MSE.t[spikes_in_attractor]/ms, MSE.i[spikes_in_attractor], 'r+')
xlabel('Time (ms)')
ylabel('Neuron index')
subplot(337)
imshow(MVI.v / mV,aspect='auto')
subplot(338)
plot(MSI.t/ms, MSI.i, '.k')
xlabel('Time (ms)')
ylabel('Neuron index')
tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

WARNING    <ipython-input-3-548a63dded37>:24: RuntimeWarning: divide by zero encountered in true_divide
  freq = 1000. * (angle(cplx[1:] / cplx[:-1])) / (2. * np.pi * (timeline[1]-timeline[0]))
 [py.warnings]
WARNING    <ipython-input-3-548a63dded37>:24: RuntimeWarning: invalid value encountered in true_divide
  freq = 1000. * (angle(cplx[1:] / cplx[:-1])) / (2. * np.pi * (timeline[1]-timeline[0]))
 [py.warnings]


In [8]:
tmin = 3002
tmax = tmin+40
sseg = spikes[time>tmin]
tseg = time[time>tmin]
sseg = sseg[tseg<tmax]
tseg = tseg[tseg<tmax]

segfreq = np.full(attractor_size,np.nan)
segphase = np.full(attractor_size,np.nan)
tstart = np.amin(tseg)
for inode in range(attractor_size):
    times = tseg[np.argwhere(sseg==inode)]
    if times.size > 1:
        segfreq[inode] = 1000 * (times.size-1) / (np.amax(times)-np.amin(times))
        segphase[inode] = (2.0e-3 * np.pi * (np.amin(times)-tstart) * segfreq[inode]) % (2*np.pi)

phasesort = np.argsort(segphase)
invphase = np.full(attractor_size,np.nan)
for idx,p in enumerate(phasesort):
    invphase[p] = idx
w3 = w2[:,phasesort][phasesort]

figure(figsize=(10,10))
subplot(221)
plot(tseg, invphase[sseg], 'k.',markersize=2)
subplot(222)
#plot(segphase)
scatter(attractor_nodes%nye,attractor_nodes//nye,c=segphase,cmap='gist_rainbow')
colorbar()
subplot(223)
hist(segfreq[~np.isnan(segfreq)],30)
#plot(segphase[phasesort],'.')
subplot(224)
imshow(w3)
colorbar()
tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
print(len(phasesort))

500


In [29]:
scheduling_summary()

object,part of,Clock dt,when,order,active
statemonitor (StateMonitor),statemonitor (StateMonitor),1. ms (every 5 steps),start,0,yes
statemonitor_2 (StateMonitor),statemonitor_2 (StateMonitor),1. ms (every 5 steps),start,0,yes
neurongroup_2_stateupdater (StateUpdater),neurongroup_2 (NeuronGroup),200. us (every step),groups,0,yes
neurongroup_3_stateupdater (StateUpdater),neurongroup_3 (NeuronGroup),200. us (every step),groups,0,yes
neurongroup_2_thresholder (Thresholder),neurongroup_2 (NeuronGroup),200. us (every step),thresholds,0,yes
neurongroup_3_thresholder (Thresholder),neurongroup_3 (NeuronGroup),200. us (every step),thresholds,0,yes
poissongroup_1_thresholder (Thresholder),poissongroup_1 (PoissonGroup),200. us (every step),thresholds,0,yes
spikemonitor (SpikeMonitor),spikemonitor (SpikeMonitor),200. us (every step),thresholds,1,yes
spikemonitor_2 (SpikeMonitor),spikemonitor_2 (SpikeMonitor),200. us (every step),thresholds,1,yes
synapses_1_pre (SynapticPathway),synapses_1 (Synapses),200. us (every step),synapses,-1,yes


In [9]:
figure(figsize=(12,4))
subplot(131)
imshow(w1)
colorbar()
subplot(132)
imshow(w2)
colorbar()
subplot(133)
plot(ds.diagonal_sums(w1))
plot(ds.diagonal_sums(w2),'r')
tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import numpy as np

N = w1.shape[0]
dest,src = np.meshgrid(range(N),range(N))
src = src.flatten()
dest = dest.flatten()
xdest = np.cos(2*np.pi*dest/N)
ydest = np.sin(2*np.pi*dest/N)
xsrc = np.cos(2*np.pi*src/N)
ysrc = np.sin(2*np.pi*src/N)

segs = np.zeros((N*N, 2, 2))
segs[:, 0, 0] = xsrc
segs[:, 0, 1] = ysrc
segs[:, 1, 0] = xdest
segs[:, 1, 1] = ydest

fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)

line_segments = LineCollection(segs, linewidths=0.01*w1.flatten())

ax.add_collection(line_segments)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
figure(figsize=(12,4))
subplot(131)
idx = 0
targets = SEE.j[idx, :] 
plot(E.x[targets] / cm, E.y[targets] / cm, 'r.')

subplot(132)
targets = SEI.j[idx, :] 
plot(I.x[targets] / cm, I.y[targets] / cm, 'r.')

subplot(133)
targets = SIE.j[idx, :] 
plot(E.x[targets] / cm, E.y[targets] / cm, 'r.')

subplot(131)
idx = 5150
targets = SEE.j[idx, :] 
plot(E.x[targets] / cm, E.y[targets] / cm, '.')

subplot(132)
targets = SEI.j[idx, :] 
plot(I.x[targets] / cm, I.y[targets] / cm, '.')

subplot(133)
idx = 820
targets = SIE.j[idx, :] 
plot(E.x[targets] / cm, E.y[targets] / cm, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

WARNING    <ipython-input-9-f9826f9c0588>:15: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  subplot(131)
 [py.warnings]
WARNING    <ipython-input-9-f9826f9c0588>:20: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  subplot(132)
 [py.warnings]
WARNING    <ipython-input-9-f9826f9c0588>:24: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future versi

In [4]:
print(np.mean(w1),np.mean(w2),w1.size,np.count_nonzero(w1),np.mean(w1)*w1.size/np.count_nonzero(w1))


0.23094016687739957 0.2412829279812967 250000 38810 1.4876331285583586


In [4]:
import pickle
time = MSE.t[spikes_in_attractor]/ms
spikes = attractor_indices_sorted[idx_in_sort]
mylist = [time,spikes]
with open('example_spikes2.pkl', 'wb') as f:
    pickle.dump(mylist, f)

In [10]:
print(defaultclock.t)

<defaultclock.t: 2.8 * second>


In [35]:
print(not STDP_immediate)

False


In [7]:
with open('params.csv', 'w') as f:
    f.write(s)

In [11]:
dir(SEE.delay)

['__add__',
 '__array__',
 '__array_prepare__',
 '__array_wrap__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 'dim',
 'dtype',
 'get_item',
 'get_subexpression_with_index_array',
 'get_with_expression',
 'get_with_index_array',
 'group',
 'group_name',
 'index_var',
 'index_var_name',
 'indexing',
 'name',
 'set_item',
 'set_with_expression',
 'set_with_expression_conditional',
 'set_with_inde

In [17]:
print(SEE.delay[323],SEE.j.shape,SEE.w.shape)

1.27121989 ms (19181504,) (19181504,)


In [17]:
print(segfreq[~np.isnan(segfreq)].T.shape)

(406,)


In [8]:
print(segfreq.shape)

(500,)
